In [ ]:
import dsar
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
from datetime import datetime
from ds_helper import load_table, combine_records, split_by_feature, get_timestamp,get_feature_importance, find_corr, cal_metrics

# timestamp = datetime.today().strftime('%Y%m%d%H%M%S')
cohort_include_smoker=False

lr_model_table = 'hb.hb_poc_premiums_rv3'
pred_out_table = 'hb.uw_sim_preds_v1'

if cohort_include_smoker:
    out_name_cohort = 's' #smoker
else:
    out_name_cohort = 'ns' #No smoker

model_id = os.environ.get('MODEL_ID', 178)

plt.style.use('ggplot')
pd.set_option('display.max_rows', 70)

In [ ]:
useful_cols = [
# 'app_age',
# 'ins_sex',
'first_pol_substandard',
'first_year',
'contract_no',
# 'annual_prem',
'adj_annual_prem',
'exposure',
'year_seq',
'risk_class',
'tot_clm_amt', 'tot_cnt', 'tot_pay_amt',
'inflate_tot_pay_amt','inflate_tot_clm_amt','inflate_tot_cnt']

columns = ', '.join(useful_cols) if useful_cols else '*'
with dsar.psql_con("HB") as con:
    df = pd.read_sql(sql=f"""select {columns} from {lr_model_table}
                            where plan_region='HK' and vhis_compatible=1
                            and year_seq<=5 and risk_class!='decline'
                            order by contract_no, year_seq
""", con = con)

df = df.drop(['tot_clm_amt', 'tot_cnt'], axis=1)
df = df.rename({
    'inflate_tot_clm_amt': 'tot_clm_amt', 
    'inflate_tot_cnt': 'tot_cnt', 
#     'inflate_tot_pay_amt': 'tot_pay_amt',
    'adj_annual_prem': 'annual_prem'
    }, axis=1)


if 'risk_class' in df:
    df['risk_class'] = df.risk_class.replace(['exclusion','exclusion_loading', 'loading'],'substandard') 
    
# NOTE: whether the sum is calculated from the raw claims or adjusted claims are determined by the variable `clm_col_end`
clm_col_end = '_clm_amt' # the ending of the claim columns. This could be '_clm_amt' or '_clm_amt_adj', depending on whether we want to use the inflation-adjusted version or not. 

unwanted_claim_cols = [
    'hom_nur_clm_amt', 'acc_clm_amt', 'smm_dr_clm_amt', 
    'cancer_clm_amt', 'smm_rb_clm_amt', 'ic_clm_amt', 
    'hi_clm_amt', 'tot_clm_amt'
    ] 

# ------------------ define extra variables for convenience ------------------ #
claim_cols = df.columns[df.columns.str.contains(clm_col_end)].to_list() # variable `claim_cols` contains all the columns related to claims

claim_cols_pruned = [col for col in claim_cols if col not in unwanted_claim_cols] # this variable removes those unwanted claim columns
# ---------------------------------------------------------------------------- #

# print some info about the dataset
print(f'n_records = {df.shape[0]}')
print(f'n_features = {df.shape[1]}')
print(f'n_claim_records = {df.tot_clm_amt.gt(0).sum()}')
print(f'n_unique_contracts = {df.contract_no.nunique()}')

In [ ]:
# the effective premium paid during the exposure period. This column is needed when making decile plots for aggregate loss ratio
df['eff_annual_prem'] = df['annual_prem'] * df['exposure']

# ---------------------------- define clean_record --------------------------- #
# A record is clean if it does not have any disease history
# df['clean_record'] = (~df[disease_groups].any(axis=1)).astype('int64')

# ----------------------------- define loss_ratio ---------------------------- # 
# there are two possible choices: either use tot_clm_amt or tot_pay_amt, but tot_pay_amt is preferred 

# df['loss_ratio'] = (df.tot_clm_amt / (df.annual_prem * df.exposure)) # use tot_clm_amt
df['loss_ratio'] = (df.tot_pay_amt / (df.annual_prem * df.exposure)) # loss ratio within the insured's exposure 
# df['loss_ratio'] = (df.tot_pay_amt/df.exposure) / df.annual_prem # loss ratio per exposure. But this is in fact the same as above

# --- define frequency and severity for the frequency*severity formulation --- #
df['frequency'] = df['tot_cnt'] / df['exposure']
df['severity'] = df['loss_ratio']/np.fmax(df['frequency'], 1)
# df['severity'] = df['loss_ratio']/np.fmax(df['tot_cnt'], 1)
# ---------------------------------------------------------------------------- #

# ---------------------- for binary classification model --------------------- #
df['has_clm'] = df['tot_cnt'].gt(0)
# ---------------------------------------------------------------------------- #

# sanity check for the calculation of frequency and severity
df.query('loss_ratio>0 and exposure !=1.0')[['exposure', 'tot_pay_amt', 'annual_prem', 'tot_cnt' , 'loss_ratio','frequency', 'severity', 'has_clm']]

In [ ]:
df_lr_mapping = combine_records(df)
df_lr_mapping = df_lr_mapping[df_lr_mapping.exposure==5]

In [ ]:
df_lr_mapping.head()

In [ ]:
df.risk_class.unique()

In [ ]:
df_lr_mapping.risk_class.unique()

In [ ]:
df_lr_mapping = df_lr_mapping.drop(['risk_class'], axis=1)

# LR model

In [ ]:
with dsar.psql_con("HB") as con:
    lr_out = pd.read_sql(sql=f"""select out.*, raw.risk_class, raw.app_age, raw.ins_sex from {pred_out_table} out                             
                            left join 
                            (select distinct contract_no, risk_class, app_age, ins_sex from {lr_model_table}
                            where risk_class is not null and year_seq=1) raw
                            on raw.contract_no = out.contract_no
                            where model_id = '{model_id}'""", con = con)

In [ ]:
lr_out

In [ ]:
lr_out = pd.merge(lr_out, df_lr_mapping, how='left', on='contract_no')

In [ ]:
lr_out = lr_out[(lr_out['risk_class']=='decline')|(lr_out['exposure']==5)]

In [ ]:
# ------------------ convert `ins_sex` to indicator variable ----------------- #
lr_out['sex_male'] = lr_out.ins_sex.map({'M':1, 'F':0})
lr_out = lr_out.drop('ins_sex', axis='columns')
    
age_gp_interval = list(range(0, 56, 5)) + [np.inf]
# age_gp_interval = [-1,10,20,30,40,50,np.inf]
# # labels = ['baby','kid','teen','adult','mid-age','senior']
lr_out['age_gp'] = pd.cut(lr_out['app_age'], age_gp_interval, right=True, include_lowest=True)

In [ ]:
lr_out.query('risk_class=="decline"')

In [ ]:
lr_out.risk_class.unique()

In [ ]:
if cohort_include_smoker: 
    cohort_col = ['age_gp','sex_male','smoker']
else:
    cohort_col = ['age_gp','sex_male']

In [ ]:
lr_out['cohort'] = lr_out[cohort_col].astype(str).agg('|'.join, axis=1)

In [ ]:
lr_out['cohort'].unique()

In [ ]:
lr_out_test = lr_out[lr_out.train.astype(int)==0]
lr_out_train = lr_out[lr_out.train.astype(int)==1]

In [ ]:
lr_out_train.groupby(cohort_col).size().reset_index()

In [ ]:
lr_out_test.groupby(cohort_col).size().reset_index()

In [ ]:
# lr_thresholds_xgb = (np.percentile(lr_out[lr_out.train_1==1]['xgb'], np.linspace(1,100,100)))
# lr_thresholds_freq = (np.percentile(lr_out[lr_out.train_1==1]['freq_severity'], np.linspace(1,100,100)))
# lr_thresholds_cla = (np.percentile(lr_out[lr_out.train_1==1]['cla_reg'], np.linspace(1,100,100)))

In [ ]:
def get_extra_class_size(df):
    decline_size = sum(df['risk_class']=='decline')
    exclusion_size = sum(df['risk_class'].isin(['exclusion', 'exclusion_loading']))
    loading_size = sum(df['risk_class'].isin(['loading', 'exclusion_loading']))
    std_size = sum(pd.to_numeric(df['first_pol_substandard'], errors='coerce')==0)
    return decline_size, exclusion_size, loading_size, std_size

def get_class_loss_stat(df):
    pay = df['tot_pay_amt'].sum()
    prem = df['eff_annual_prem'].sum()
    size = len(df)
    return pay, prem, size

In [ ]:
def map_cohort_percentile(lr_out_test,lr_out_train, pred_col_name):
    '''
    Returns dataframe with loss ratio base on different loss ratio threshold by cohort.

            Parameters:
                    lr_out_test (pd.DataFrame): test set with output for loss model
                    lr_out_train (pd.DataFrame): train set with output for loss model
                    pred_col_name (str): column name for predicted value (predicted loss ratio)
                    true_col_name (str): column name for true label (true loss ratio)
            Returns:
                    result of different thresholds
                    threshold for each cohort
    '''
    cohort_threshold_dict={}
    lr_cohort_test = pd.DataFrame()
    lr_cohort_train = pd.DataFrame()
    for cohort in lr_out_test['cohort'].unique(): #loop each cohort
        lr_subset_train = lr_out_train.loc[lr_out_train['cohort']==cohort].copy()
        lr_subset_test = lr_out_test.loc[lr_out_test['cohort']==cohort].copy()
        if len(lr_subset_test)==0 or len(lr_subset_train)==0:
            continue
        else:
            lr_thresholds = np.percentile(lr_subset_train[pred_col_name],
                                          np.linspace(1,99,99)) #find percentile from train
            lr_subset_test.loc[:,'cohort_percentile'] = np.digitize(lr_subset_test[pred_col_name],
                                                              lr_thresholds, right=True)+1 #map percentile to test
            lr_subset_train.loc[:,'cohort_percentile'] = np.digitize(lr_subset_train[pred_col_name],
                                                              lr_thresholds, right=True)+1 #map percentile to train
            lr_cohort_test = pd.concat([lr_cohort_test,lr_subset_test], axis=0) #merge all cohort together
            lr_cohort_train = pd.concat([lr_cohort_train,lr_subset_train], axis=0) #merge all cohort together
            cohort_threshold_dict[cohort] = lr_thresholds
    
    return lr_cohort_train,lr_cohort_test, cohort_threshold_dict

In [ ]:
lr_out_train, lr_out_test,cohort_threshold_dict = map_cohort_percentile(lr_out_test,lr_out_train, 'pred')

In [ ]:
lr_out_train.columns

In [ ]:
def map_percentile_class(srs, pref_percentile, std_percentile):
    if srs<=pref_percentile:
        return 'pref'
    elif srs<=std_percentile:
        return 'std'
    else:
        return 'substd'

In [ ]:
lr_out_test['class'] = lr_out_test['cohort_percentile'].apply(map_percentile_class, args=(10,91))

In [ ]:
age_gp_interval = list(range(0, 56, 10)) + [np.inf]
age_gp_interval

In [ ]:
age_gp_interval_origin = list(range(0, 56, 5)) + [np.inf]
age_gp_interval_large = [0, 20, 30, 40, 50, np.inf]

In [ ]:
def twin_lineplot(x,y,**kwargs):
    ax = plt.twinx()
    sns.pointplot(x=x,y=y,**kwargs, ax=ax)
    ax.grid(visible=False)
    ax.set_ylabel('record count')

        
def plot_all(df_lr, grpby_sex, age_bin=age_gp_interval_origin, model_id=model_id, out=False):
    if out:
        directory = f'./model_{model_id}'
        if not os.path.exists(directory):
            os.makedirs(directory)
    df = df_lr
    hasgender = 'Y' if grpby_sex else 'N'
    df['age_gp'] = pd.cut(df['app_age'], age_bin, right=True, include_lowest=True)
    unique_bin_n = df['age_gp'].nunique()
    df['n_record'] = 1
    df['decline_cnt'] = (df.risk_class=='decline').astype(int)
    by = ['age_gp','class']
    if grpby_sex:
        by.append('sex_male')
    diff_df = df.groupby(by)[['tot_pay_amt','eff_annual_prem','n_record']].sum().assign(
                                loss_ratio = lambda df: df['tot_pay_amt']/df['eff_annual_prem']).unstack(1)
    diff_df['pref_std_diff'] = (diff_df['loss_ratio']['pref']/diff_df['loss_ratio']['std']-1)
    diff_df['std_substd_diff'] = (diff_df['loss_ratio']['std']/diff_df['loss_ratio']['substd']-1)
    by_no_class = [col for col in by if col !='class']
    diff_df.columns = diff_df.columns.map(lambda x: '_'.join(x).strip('_'))
#     diff_df['n_record'] = diff_df[['n_record_pref', 'n_record_std', 'n_record_substd']].sum(axis=1)
    diff_df = diff_df.reset_index()
    by_no_class = [col for col in by if col !='class']
    diff_df = pd.melt(diff_df, id_vars=by_no_class + ['n_record_pref', 'n_record_std', 'n_record_substd'],
                      value_vars=['pref_std_diff','std_substd_diff'],
        var_name='diff_type', value_name='prop_diff', ignore_index=False)
    diff_df['n_record'] = diff_df.apply(lambda x:
                                   x['n_record_pref'] + x['n_record_std'] if x['diff_type'] == 'pref_std_diff'
                                  else x['n_record_substd'] + x['n_record_std'], axis=1)
    plot_df = df.groupby(by)[['decline_cnt','n_record','tot_pay_amt','eff_annual_prem']].sum().assign(
        decline_prop = lambda df: df['decline_cnt']/df['n_record'],
        loss_ratio = lambda df: df['tot_pay_amt']/df['eff_annual_prem']).reset_index()
    
    g = sns.catplot(data=plot_df, x='age_gp', y='decline_prop', row='sex_male' if grpby_sex else None,
                    col='class', kind='bar', sharex=False, sharey=True,)
    g.map(twin_lineplot,'age_gp','n_record')
    g.set_xticklabels(rotation=30, ha="right")
    g.set_ylabels('decline ratio')
    g.figure.suptitle('decline ratio')
    g.tight_layout() 
    if out:
        g.savefig(os.path.join(directory,f'model{model_id}_bin{unique_bin_n}_gender{hasgender}_decline.jpg'), dpi = 250)
    g = sns.catplot(data=plot_df, x='age_gp', y='loss_ratio', row='sex_male' if grpby_sex else None,
                    col='class', kind='bar', sharex=False, sharey=True)
    g.map(twin_lineplot,'age_gp','n_record')
    g.set_xticklabels(rotation=30, ha="right")
    g.set_ylabels('loss ratio')
    g.figure.suptitle('loss ratio')
    g.tight_layout() 
    if out:
        g.savefig(os.path.join(directory,f'model{model_id}_bin{unique_bin_n}_gender{hasgender}_loss.jpg'), dpi = 250)
        
    g = sns.catplot(data=diff_df, x='age_gp', y='prop_diff', row='sex_male' if grpby_sex else None,
               col = 'diff_type',
               kind='bar',
               sharex=False, sharey=True)
    g.map(twin_lineplot,'age_gp','n_record')
    g.set_ylabels('class risk differentiation')
    g.set_xticklabels(rotation=30, ha="right")
    g.figure.suptitle('class risk differentiation')
    g.tight_layout() 
    if out:
        g.savefig(os.path.join(directory,f'model{model_id}_bin{unique_bin_n}_gender{hasgender}_propdiff.jpg'), dpi = 250)

In [ ]:
plot_all(lr_out_test, grpby_sex = False , age_bin = age_gp_interval_large, model_id=model_id, out=True)

In [ ]:
plot_all(lr_out_test, grpby_sex = False , age_bin = age_gp_interval_origin, model_id=model_id, out=True)

In [ ]:
plot_all(lr_out_test, grpby_sex = True , age_bin = age_gp_interval_large, model_id=model_id, out=True)

In [ ]:
plot_all(lr_out_test, grpby_sex = True , age_bin = age_gp_interval_origin, model_id=model_id, out=True)

In [ ]:
lr_out_test.groupby(['age_gp']).count()

In [ ]:
diff_df = lr_out_test.groupby(['age_gp','class','sex_male'])[['tot_pay_amt','eff_annual_prem', 'n_record']].sum().assign(
                                loss_ratio = lambda df: df['tot_pay_amt']/df['eff_annual_prem']).unstack(1)
diff_df['pref_std_diff'] = (diff_df['loss_ratio']['pref']/diff_df['loss_ratio']['std']-1)
diff_df['std_substd_diff'] = (diff_df['loss_ratio']['std']/diff_df['loss_ratio']['substd']-1)
by_no_class = [col for col in ['age_gp','class','sex_male'] if col !='class']
diff_df.columns = diff_df.columns.map(lambda x: '_'.join(x).strip('_'))
diff_df['n_record'] = diff_df[['n_record_pref', 'n_record_std', 'n_record_substd']].sum(axis=1)
diff_df = diff_df.reset_index()
# diff_df = diff_df[['age_gp','sex_male','pref_std_diff','std_substd_diff']].droplevel(1, axis=1) 
diff_df = pd.melt(diff_df, id_vars=['age_gp','sex_male']+ ['n_record'], value_vars=['pref_std_diff','std_substd_diff'],
        var_name='diff_type', value_name='prop_diff', ignore_index=False)
diff_df

In [ ]:
def get_threshold_range(lr_out_test, pref_min, pref_max, std_min, std_max):
    test_stat = lr_out_test.cohort_percentile.value_counts(normalize=True).rename('proportion').sort_index().reset_index()
    test_stat['cum_prop'] = test_stat['proportion'].cumsum()
    test_stat = test_stat.rename(columns={'index':'percentile'})
    pref_stat = test_stat.query(f'{pref_min}<=cum_prop<={pref_max}')
    pref_min_p, pref_max_p = pref_stat['percentile'].min(), pref_stat['percentile'].max()
    std_stat = test_stat.query(f'{std_min}<=cum_prop<={std_max}')
    std_min_p, std_max_p = std_stat['percentile'].min(), std_stat['percentile'].max()
    return pref_min_p, pref_max_p, std_min_p, std_max_p

In [ ]:
pref_min_p, pref_max_p, std_min_p, std_max_p  = get_threshold_range(lr_out_test, 0.07, 0.13, 0.87, 0.93)
# pref_min_p, pref_max_p, std_min_p, std_max_p  = get_threshold_range(lr_out_test, 0.14, 0.13, 0.87, 0.93)

In [ ]:
pref_min_p, pref_max_p, std_min_p, std_max_p 

In [ ]:
def evaluate_lr_threshold(lr_out_test, pred_col_name,pref_min_p,pref_max_p, std_min_p, std_max_p):
    lr_evaluation=[]
    for i in range(pref_min_p,pref_max_p+1):
        pref_percentile = i
        pref = lr_out_test[lr_out_test[pred_col_name]<=i]
        pref_pay, pref_prem, pref_size = get_class_loss_stat(pref)
        pref_decline, pref_exclusion, pref_loading, pref_std = get_extra_class_size(pref)
        for j in range(max(std_min_p,pref_max_p+1),std_max_p+1):
            std_percentile = j
            std = lr_out_test[(lr_out_test[pred_col_name]>i) & 
                                     (lr_out_test[pred_col_name]<=j)]
            std_pay, std_prem, std_size = get_class_loss_stat(std)
            std_decline, std_exclusion, std_loading, std_std = get_extra_class_size(std)
            S_P_test = lr_out_test[lr_out_test[pred_col_name]<=j]
            if len(S_P_test)==0:
                agreement_rate=0
            else:
                agreement_rate = sum(pd.to_numeric(S_P_test['first_pol_substandard'], errors='coerce')==0)/len(S_P_test)
            
            substd = lr_out_test[(lr_out_test[pred_col_name]>j)]
            substd_pay, substd_prem, substd_size = get_class_loss_stat(substd)
            substd_decline, substd_exclusion, substd_loading, substd_std = get_extra_class_size(substd)
            if len(substd)==0:
                substd_agreement_rate=0
            else:
                substd_agreement_rate = sum(pd.to_numeric(substd['first_pol_substandard'], errors='coerce')!=0)/len(substd)
            threshold_result = [pref_percentile, std_percentile, pref_size,
                                std_size,substd_size, 
                                agreement_rate, substd_agreement_rate,
                               pref_decline, pref_exclusion, pref_loading,
                               std_decline, std_exclusion, std_loading,
                               substd_decline, substd_exclusion, substd_loading,
                               pref_pay,std_pay,substd_pay,
                               pref_prem, std_prem, substd_prem,
                               pref_std, std_std, substd_std]
            lr_evaluation.append(threshold_result)

    lr_evaluation = pd.DataFrame(lr_evaluation, columns = ['pref_percentile','std_percentile',
                                                           'pref_size','std_size','substd_size',
                                                           'agreement_rate','nstd_agreement_rate',
                                                          'pref_decline', 'pref_exclusion', 'pref_loading',
                                                          'std_decline', 'std_exclusion', 'std_loading',
                                                          'substd_decline', 'substd_exclusion', 'substd_loading',
                                                          'pref_pay','std_pay','substd_pay',
                                                           'pref_prem', 'std_prem', 'substd_prem',
                                                          'pref_std', 'std_std', 'substd_std'])  
    return lr_evaluation

In [ ]:
def post_process_lr_eva(lr_eva):
    table = lr_eva
    table['pref_prop'] = table['pref_size']/table[['pref_size','std_size','substd_size']].sum(axis=1)
    table['std_prop'] = table['std_size']/table[['pref_size','std_size','substd_size']].sum(axis=1)
    table['substd_prop'] = table['substd_size']/table[['pref_size','std_size','substd_size']].sum(axis=1)
#     pref_size = (table['pref_size']-table['pref_decline'])
#     std_size = (table['std_size']-table['std_decline'])
#     substd_size = (table['substd_size']-table['substd_decline'])
    for risk_class in ['pref', 'std', 'substd']:
        table[f'{risk_class}_loss'] = (table[f'{risk_class}_pay']).divide(table[f'{risk_class}_prem'])
        table[f'{risk_class}_decline_prop'] = (table[f'{risk_class}_decline']).divide(table[f'{risk_class}_size'])
    table['S_P_loss'] = (table[f'pref_pay'] +
                         table[f'std_pay']).divide(table[f'pref_prem']+
                                                             table[f'std_prem'])
    return table

In [ ]:
def parse_threshold_to_df(threshold_dict): 
    threshold_df = pd.DataFrame()
    for k, v in threshold_dict.items():
        cohort_threshold = pd.Series(v).reset_index()
        cohort_threshold.columns =['precentile', 'threshold']
        cohort_threshold['precentile'] = cohort_threshold['precentile']+1
        cohort_threshold['cohort'] = k
        threshold_df = pd.concat([threshold_df,cohort_threshold], axis=0)
    threshold_df[cohort_col] =   threshold_df['cohort'].str.split('|',expand=True)
    threshold_df = threshold_df.drop('cohort', axis=1)
    return threshold_df

In [ ]:
out_col = ['model_id', 'pref_percentile', 'std_percentile', 'pref_prop', 'std_prop','substd_prop',
           'pref_loss', 'std_loss',
           'substd_loss','S_P_loss','agreement_rate', 'nstd_agreement_rate',
        'pref_decline_prop', 'std_decline_prop', 'substd_decline_prop',
        'pref_size', 'std_size','substd_size',
       'pref_decline', 'pref_exclusion', 'pref_loading',
      'std_decline', 'std_exclusion', 'std_loading',
    'substd_decline', 'substd_exclusion', 'substd_loading',
    'pref_std', 'std_std', 'substd_std'
]

In [ ]:
# if not pd.Series([pref_min_p,pref_max_p, std_min_p, std_max_p]).isnull().any():
#     eva_out = evaluate_lr_threshold(lr_out_test, 'cohort_percentile',pref_min_p,pref_max_p, std_min_p, std_max_p)
#     eva_out['model_id'] = model_id
#     eva_out = post_process_lr_eva(eva_out)
#     cohort_threshold_df =  parse_threshold_to_df(cohort_threshold_dict)
#     cohort_threshold_df['model_id'] =  model_id
#     with dsar.psql_con('HB') as con:
#         eva_out[out_col].to_sql(name=f'uw_sim_tune', con=con, schema='hb',index=False, if_exists='append')
#     #     cohort_threshold_df.to_sql(name=f'{out_name}', con=con, schema='hb',index=False, if_exists='append')
# else:
#     os.write(1, f"model_id: {model_id}, no suitable threshold found\n".encode())

In [ ]:
eva_out